In [1]:
import numpy
import matplotlib
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
!nvidia-smi


Sat Dec 17 09:26:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 527.37       Driver Version: 527.37       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:27:00.0  On |                  N/A |
|  0%   46C    P5    19W / 125W |    960MiB /  6144MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
import torch.nn as nn
import torch
import torch.optim as optim
import torch.nn.functional as F

from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset

from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import glob
import random

learning_rate = 0.001
batch_size = 10
epochs = 10

#устанавливаем зерно и случайное значение
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1234)
if device =='cuda':
    torch.cuda.manual_seed_all(1234)


print(device)

Task 1-2

In [ ]:
train_path = 'D:\Git\pyLab5_3sem\\train'
test_path = 'D:\Git\pyLab5_3sem\\test'
val_path = 'D:\Git\pyLab5_3sem\\val'

path1 = "dataset/zebra/zebra_annotation.csv"
path2 = "dataset/bay horse/bay horse_annotation.csv"
dfs = []
tmp = pd.read_csv(path1, sep=',', header=None)
dfs.append(tmp)
tmp = pd.read_csv(path2, sep=',', header=None)
dfs.append(tmp)
df = pd.concat(dfs)
df.drop(1, axis=1, inplace=True)
df.rename(columns={0: 'absolute_way', 2: 'class_img'}, inplace=True)
df.reset_index(inplace=True)


if not os.path.isdir(train_path):
    os.mkdir(train_path)

if not os.path.isdir(test_path):
    os.mkdir(test_path)

if not os.path.isdir(val_path):
    os.mkdir(val_path)


def load_train(df: pd.core.frame.DataFrame, path: str, i: int) -> None:
    '''
    dowload img from dataframe with path
    '''
    image_path = df.absolute_way[i]
    image = cv2.imread(image_path)
    cv2.imwrite(os.path.join(path, f'{df.class_img[i]}.{i}.jpg'), image)


def load_test(df: pd.core.frame.DataFrame, path: str, i: int) -> None:
    '''
    dowload img from dataframe with path, work with separation 80:10:10
    '''
    image_path = df.absolute_way[i]
    image = cv2.imread(image_path)
    if i - 840 > 104:
        i = i - 1785
    else:
        i = i - 840
    cv2.imwrite(os.path.join(path, f'{i}.jpg'), image)


# using
for i in range(840, 945):
    load_test(df, test_path, i)

for i in range(945, 1050):
    load_train(df, val_path, i)

for i in range(1050, 1890):
    load_train(df, train_path, i)

for i in range(1890, 1995):
    load_test(df, test_path, i)

for i in range(1995, 2100):
    load_train(df, val_path, i)


train_list = glob.glob(os.path.join(train_path, '*.jpg'))
test_list = glob.glob(os.path.join(test_path, '*.jpg'))

train_list, val_list = train_test_split(train_list, test_size=0.1)
